# Guitar Effects Processor
This notebook implements a real-time audio processing system for guitar effects with a modular design an

In [ ]:
# Cell 1: Import statements and setup for all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
import scipy.signal as signal
import threading
import time
import queue
import gradio as gr
from typing import List, Dict, Callable, Any

In [ ]:
# Cell 2: Define a modular effects processor class that can chain multiple effects

class EffectsProcessor:
    def __init__(self):
        self.effects = []
        self.enabled_effects = []
        self.sample_rate = 44100
        self.block_size = 1024
        self.input_queue = queue.Queue()
        self.output_queue = queue.Queue()
        self.running = False
        self.input_stream = None
        self.output_stream = None
        self.processing_thread = None
    
    def add_effect(self, effect, enabled=True):
        """Add an effect to the chain"""
        self.effects.append(effect)
        self.enabled_effects.append(enabled)
        return len(self.effects) - 1  # Return the index of the added effect
    
    def remove_effect(self, index):
        """Remove an effect from the chain"""
        if 0 <= index < len(self.effects):
            self.effects.pop(index)
            self.enabled_effects.pop(index)
            return True
        return False
    
    def toggle_effect(self, index, enabled=None):
        """Enable/disable an effect"""
        if 0 <= index < len(self.effects):
            if enabled is None:
                self.enabled_effects[index] = not self.enabled_effects[index]
            else:
                self.enabled_effects[index] = enabled
            return True
        return False
    
    def reorder_effects(self, new_order):
        """Reorder the effects chain"""
        if len(new_order) != len(self.effects):
            return False
        
        new_effects = []
        new_enabled = []
        
        try:
            for i in new_order:
                new_effects.append(self.effects[i])
                new_enabled.append(self.enabled_effects[i])
                
            self.effects = new_effects
            self.enabled_effects = new_enabled
            return True
        except IndexError:
            return False
    
    def process_audio(self, indata, frames, time, status):
        """Callback for audio input"""
        self.input_queue.put(indata.copy())
    
    def get_audio(self, outdata, frames, time, status):
        """Callback for audio output"""
        try:
            outdata[:] = self.output_queue.get_nowait()
        except queue.Empty:
            outdata.fill(0)
    
    def process_thread(self):
        """Processing thread that applies effects"""
        while self.running:
            try:
                # Get input audio block
                audio_block = self.input_queue.get(timeout=0.1)
                
                # Apply each enabled effect
                for i, effect in enumerate(self.effects):
                    if self.enabled_effects[i]:
                        audio_block = effect.process(audio_block)
                
                # Send to output
                self.output_queue.put(audio_block)
                
            except queue.Empty:
                continue
    
    def start(self):
        """Start audio processing"""
        if self.running:
            return False
        
        self.running = True
        
        # Start processing thread
        self.processing_thread = threading.Thread(target=self.process_thread)
        self.processing_thread.daemon = True
        self.processing_thread.start()